## DGL中的DeepWalk和Node2Vec

这里我们讲解DGL里实现DeepWalk和Node2Vec的方法。值得注意的是，DeepWalk可以看作是参数p和参数q为1的Node2Vec模型。

这里按照图嵌入通用框架的四个部分，信息提取器、映射函数、重构器和优化目标，来实现Node2Vec。

具体地，信息提取器就是提取随机游走，映射函数就是一层Embedding Layer，重构器和优化目标即是由节点嵌入的内积构成的函数。

下面这部分代码来自DGL样例中的[node2vec.py](https://github.com/dmlc/dgl/blob/master/examples/pytorch/node2vec/model.py)，这里做了一点小修改来让它更简洁。

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import dgl
from dgl.data import CoraGraphDataset
from dgl.sampling import node2vec_random_walk
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score

/home/studio-lab-user/.conda/envs/dgl/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Node2Vec(nn.Module):

    ################### PART 1 ###################
    def __init__(self, g, embedding_dim, walk_length, p, q, num_walks=10, window_size=5, num_negatives=5,
                 use_sparse=True, weight_name=None):
        super(Node2Vec, self).__init__()

        assert walk_length >= window_size

        self.g = g
        
        # 一些基本的超参数
        self.embedding_dim = embedding_dim
        self.walk_length = walk_length
        self.p = p
        self.q = q
        self.num_walks = num_walks
        self.window_size = window_size
        self.num_negatives = num_negatives
        self.N = self.g.num_nodes()  # 节点数量
        if weight_name is not None:
            self.prob = weight_name
        else:
            self.prob = None

        # 初始化映射函数里的参数
        self.embedding = nn.Embedding(self.N, embedding_dim, sparse=use_sparse)

    def reset_parameters(self):
        self.embedding.reset_parameters()

    def sample(self, batch):
        """
        构建正和负样本。
        正样本来自随机游走；
        负样本来自随机采样。
        """
        # batch其实就是一组节点
        if not isinstance(batch, torch.Tensor):
            batch = torch.tensor(batch)

        batch = batch.repeat(self.num_walks)    # 直接复制节点，完成重复次数的计算
        
        # 采样正样本，使用DGL的random walk的方法
        pos_traces = node2vec_random_walk(self.g, batch, self.p, self.q, self.walk_length, self.prob)
        pos_traces = pos_traces.unfold(1, self.window_size, 1)    # 用窗口划分得到的随机游走
        pos_traces = pos_traces.contiguous().view(-1, self.window_size)

        # 采样负样本
        neg_batch = batch.repeat(self.num_negatives)    # 直接复制节点，完成重复次数的计算
        neg_traces = torch.randint(self.N, (neg_batch.size(0), self.walk_length))    # 直接随机采样图中节点序列作为负样本
        neg_traces = torch.cat([neg_batch.view(-1, 1), neg_traces], dim=-1)
        neg_traces = neg_traces.unfold(1, self.window_size, 1)    # 用窗口划分得到的随机游走
        neg_traces = neg_traces.contiguous().view(-1, self.window_size)

        return pos_traces, neg_traces

    def forward(self, nodes=None):
        """
        返回输入节点的Embedding
        """
        emb = self.embedding.weight
        if nodes is None:
            return emb
        else:
            return emb[nodes]

    def loss(self, pos_trace, neg_trace):
        """根据正负样本计算损失函数"""
        e = 1e-15  # 出现在下面的地方，用于防止torch.log()中的输入为0

        # 正样本的损失函数
        pos_start, pos_rest = pos_trace[:, 0], pos_trace[:, 1:].contiguous()
        w_start = self.embedding(pos_start).unsqueeze(dim=1)
        w_rest = self.embedding(pos_rest)
        pos_out = (w_start * w_rest).sum(dim=-1).view(-1)  # 计算内积

        # 负样本的损失函数
        neg_start, neg_rest = neg_trace[:, 0], neg_trace[:, 1:].contiguous()

        w_start = self.embedding(neg_start).unsqueeze(dim=1)
        w_rest = self.embedding(neg_rest)
        neg_out = (w_start * w_rest).sum(dim=-1).view(-1)  # 计算内积

        # 计算损失值
        pos_loss = -torch.log(torch.sigmoid(pos_out) + e).mean()  # 希望start, rest的嵌入的内积大
        neg_loss = -torch.log(1 - torch.sigmoid(neg_out) + e).mean()  # 希望start, rest的嵌入的内积小

        return pos_loss + neg_loss
    
    def loader(self, batch_size):

        return DataLoader(torch.arange(self.N), batch_size=batch_size, shuffle=True, collate_fn=self.sample)
    

In [3]:
# 训练Node2Vec模型
def train(model, loader, optimizer, device):
    model.train()
    model = model.to(device)
    total_loss = 0
    for pos_traces, neg_traces in loader:
        pos_traces, neg_traces = pos_traces.to(device), neg_traces.to(device)
        optimizer.zero_grad()
        loss = model.loss(pos_traces, neg_traces)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [4]:
@torch.no_grad()
def test(model, g):
    model.eval()
    z = model()
    z = z.detach().numpy() # detach()将tensor从计算图中脱离出来，numpy()把tensor转换成numpy.array格式
    _, acc = evaluate_node_classification(z, g.ndata['label'], g.ndata['train_mask'], g.ndata['test_mask'], max_iter=150)
    return acc

In [5]:
# 为衡量生成的图嵌入的质量，我们可以训练一个线性模型（比如逻辑回归模型）来预测节点的标签
def evaluate_node_classification(embedding_matrix, labels, train_mask, 
                                 test_mask, normalize_embedding=True, max_iter=1000):

    if normalize_embedding:
        embedding_matrix = normalize(embedding_matrix)

    features_train = embedding_matrix[train_mask]
    features_test = embedding_matrix[test_mask]
    labels_train = labels[train_mask]
    labels_test = labels[test_mask]


    clf = LogisticRegression(solver='lbfgs', max_iter=max_iter, multi_class='auto')
    clf.fit(features_train, labels_train)

    preds = clf.predict(features_test)
    test_acc = accuracy_score(labels_test, preds)
    return preds, test_acc

In [6]:
dataset = CoraGraphDataset('./data') # 将数据保存在data文件夹下
g = dataset[0]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Node2Vec(g, embedding_dim=128, walk_length=20,
                 p=1, q=1, num_walks=10, window_size=10,
                 num_negatives=5, use_sparse=True).to(device)

loader = model.loader(batch_size=128) # 提取共现信息
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

for epoch in range(10): # 这里使用更大的epoch将提高性能，比如100
    loss = train(model, loader, optimizer, device)
    acc = test(model, g)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Acc: {acc:.4f}')

Epoch: 01, Loss: 7.9236, Acc: 0.1590
Epoch: 02, Loss: 5.7525, Acc: 0.1970
Epoch: 03, Loss: 4.5620, Acc: 0.2390
Epoch: 04, Loss: 3.6935, Acc: 0.2760
Epoch: 05, Loss: 3.0286, Acc: 0.3390
Epoch: 06, Loss: 2.5149, Acc: 0.3840
Epoch: 07, Loss: 2.1284, Acc: 0.4320
Epoch: 08, Loss: 1.8396, Acc: 0.4620
Epoch: 09, Loss: 1.6125, Acc: 0.5010
